In [ ]:
# Importação dos módulos necessários

import models
import queries
import utils

import geopandas as gpd
import logging
import networkx as nx
import osmnx as ox
import pandas as pd
import sqlalchemy
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning) 

In [ ]:
# Conexão com o banco de dados

EXEC_MODO_TESTES = True
dados_conexao = models.ConexaoBD(EXEC_MODO_TESTES)

engine = sqlalchemy.create_engine(f'postgresql://{dados_conexao.user}:{dados_conexao.password}@{dados_conexao.server}:{dados_conexao.port}/{dados_conexao.database}')
conexao_bd = engine.connect()

In [ ]:
# Criação das tabelas no banco de dados

try :
    conexao_bd.execute(queries.CREATE_TABELA_MATRIZ_TEMPO_VIAGEM)
except Exception as e:
    logging.exception(msg='Erro inesperado ao criar tabelas no banco de dados')

In [ ]:
# Geração das associações entre pontos de origem (centróides de hexágonos) e pontos de destino (pontos de interesse)

def gerar_associacoes_origens_destinos(gdf_malha_hexagonal:gpd.GeoDataFrame, gdf_pontos_interesse:gpd.GeoDataFrame, gdf_areas_analise:gpd.GeoDataFrame):
    gdf_malha_metrico = gdf_malha_hexagonal.to_crs('EPSG:3857')
    gdf_pontos_metrico = gdf_pontos_interesse.to_crs('EPSG:3857')
    gdf_areas_metrico = gdf_areas_analise.to_crs('EPSG:3857')

    df_malha_area_analise = pd.merge(left=gdf_malha_metrico, right=gdf_areas_metrico, how='inner', left_on='codigo', right_on='codigo_hexagono', suffixes=('_h', '_aa'))
    df_origens_destinos = pd.DataFrame(data=[], columns=['codigo_origem', 'origem', 'codigo_destino', 'destino'])

    for idx in df_malha_area_analise.index:
        codigo_origem = df_malha_area_analise['codigo_h'][idx]
        origem = df_malha_area_analise['geometria_h'][idx].centroid

        flag_pontos_relevantes = gdf_pontos_metrico.centroid.within(df_malha_area_analise['geometria_aa'][idx])
        gdf_pontos_relevantes = gdf_pontos_metrico[flag_pontos_relevantes]

        for idx_pr in gdf_pontos_relevantes.index:
            df_origens_destinos.loc[len(df_origens_destinos)] = [
                codigo_origem,
                origem,
                gdf_pontos_relevantes['codigo'][idx_pr],
                gdf_pontos_relevantes['geometria'][idx_pr]
            ]

    gdf_origens_destinos = gpd.GeoDataFrame(data=df_origens_destinos, geometry='origem', crs='EPSG:3857').to_crs('EPSG:4326')
    gdf_origens_destinos = gdf_origens_destinos.set_geometry('destino').set_crs('EPSG:3857').to_crs('EPSG:4326')
    return gdf_origens_destinos

In [ ]:
# Cálculo da matriz do tempo de viagem

def calcular_matriz_tempo_viagem(gdf_origens_destinos:gpd.GeoDataFrame, gph_rede_transporte:nx.MultiDiGraph):
    df_matriz_tempo_viagem = pd.DataFrame(data=[], columns=['codigo_origem', 'no_origem', 'origem', 'codigo_destino', 'no_destino', 'destino', 'tempo_viagem_seg', 'distancia_m', 'rota'])

    for idx in gdf_origens_destinos.index:
        ponto_origem = gdf_origens_destinos['origem'][idx]
        ponto_destino = gdf_origens_destinos['destino'][idx]

        no_origem = ox.nearest_nodes(gph_rede_transporte, ponto_origem.coords[0][0], ponto_origem.coords[0][1])
        no_destino = ox.nearest_nodes(gph_rede_transporte, ponto_destino.coords[0][0], ponto_destino.coords[0][1])
        
        if no_origem != no_destino:
            try:
                rota = ox.shortest_path(gph_rede_transporte, no_origem, no_destino, weight='travel_time', cpus=2)
            except Exception as e:
                logging.exception(f'Erro inesperado ao buscar a rota para os nós {no_origem} e {no_destino}')
                continue

            gdf_rota = ox.utils_graph.route_to_gdf(gph_rede_transporte, rota, 'travel_time')
            tempo_viagem_seg = round((gdf_rota['travel_time'].sum() / 60), 2)
            distancia_m = round(gdf_rota['length'].sum(), 2)
        else:
            tempo_viagem_seg = 0
            distancia_m = 0

        df_matriz_tempo_viagem.loc[len(df_matriz_tempo_viagem)] = [
            gdf_origens_destinos['codigo_origem'][idx],
            no_origem,
            ponto_origem,
            gdf_origens_destinos['codigo_destino'][idx],
            no_destino,
            ponto_destino,
            tempo_viagem_seg,
            distancia_m,
            rota
        ]

    return gpd.GeoDataFrame(data=df_matriz_tempo_viagem, geometry='origem', crs='EPSG:4326')

In [ ]:
# Realização da análise de acessibilidade urbana dos municípios com dados suficientes

df_municipios = pd.read_sql(sql=queries.SELECT_MUNICIPIOS_APTOS_ANALISE, con=conexao_bd)
df_modalidade_transporte = pd.read_sql(sql=queries.SELECT_MODALIDADES_TRANSPORTE, con=conexao_bd)

for idx in tqdm(iterable=df_municipios.index, desc='Calculando a matriz de tempo de viagem dos municípios'):
    codigo_municipio = df_municipios['codigo'][idx]

    query = utils.montar_query_parametrizada(sql=queries.SELECT_MALHA_MUNICIPIO, params={'codigo_municipio':codigo_municipio})
    gdf_malha_hexagonal = gpd.read_postgis(sql=query, con=conexao_bd, geom_col='geometria', crs='EPSG:4326')

    query = utils.montar_query_parametrizada(sql=queries.SELECT_PONTOS_INTERESSE_MUNICIPIO, params={'codigo_municipio':codigo_municipio})
    gdf_pontos_interesse = gpd.read_postgis(sql=query, con=conexao_bd, geom_col='geometria', crs='EPSG:4326')

    for idx_m in df_modalidade_transporte.index:

        codigo_modalidade_transporte = df_modalidade_transporte['codigo'][idx_m]

        query = utils.montar_query_parametrizada(sql=queries.SELECT_AREAS_ANALISE_MUNICIPIO, params={'codigo_municipio':codigo_municipio, 'codigo_modalidade_transporte':codigo_modalidade_transporte})
        gdf_areas_analise = gpd.read_postgis(sql=query, con=conexao_bd, geom_col='geometria', crs='EPSG:4326')

        query = utils.montar_query_parametrizada(sql=queries.SELECT_GRAFO_NOS, params={'codigo_municipio':codigo_municipio, 'codigo_modalidade_transporte':codigo_modalidade_transporte})
        gdf_nodes = gpd.read_postgis(sql=query, con=conexao_bd, geom_col='geometry', crs='EPSG:4326', index_col='osmid')

        query = utils.montar_query_parametrizada(sql=queries.SELECT_GRAFO_ARESTAS, params={'codigo_municipio':codigo_municipio, 'codigo_modalidade_transporte':codigo_modalidade_transporte})
        gdf_edges = gpd.read_postgis(sql=query, con=conexao_bd, geom_col='geometry', crs='EPSG:4326', index_col=['u', 'v', 'key'])

        gph_rede_transporte = ox.graph_from_gdfs(gdf_nodes, gdf_edges)

        gdf_origens_destinos = gerar_associacoes_origens_destinos(gdf_malha_hexagonal, gdf_pontos_interesse, gdf_areas_analise)
        gdf_matriz_tempo_viagem = calcular_matriz_tempo_viagem(gdf_origens_destinos, gph_rede_transporte)
        gdf_matriz_tempo_viagem['codigo_municipio'] = df_municipios['codigo'][idx]
        gdf_matriz_tempo_viagem['codigo_modalidade_transporte'] = df_modalidade_transporte['codigo'][idx_m]

        gdf_matriz_tempo_viagem.to_postgis(name='t_matriz_tempo_viagem', con=conexao_bd, schema='public', if_exists='append', index=False)